In [5]:
from brian2 import *
import numpy as np

start_scope()
defaultclock.dt = 0.0001*ms

# Your custom timing function (unchanged)
@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def spike_timing(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    if w >= 0:
        return x**(1 - w)
    else:
        return 1 - (1 - x)**(1 + w)

def mini_urd(inputs, weights_1):
    n_input  = 2
    n_hidden = 1
    n_total  = n_input + n_hidden

    neurons = NeuronGroup(
        n_total,
        '''
        v               : 1
        sum             : 1
        spikes_received : 1
        scheduled_time  : second
        global_clock    : 1
        ''',
        threshold='v > 1',
        reset='v = 0',
        method='exact'
    )
    neurons.v = 0
    neurons.sum = 0
    neurons.spikes_received = 0
    neurons.global_clock = 0
    neurons.scheduled_time = 1e9*second

    # 2‐neuron stimulus
    stim = SpikeGeneratorGroup(2,
        indices=[0,1],
        times=inputs*ms
    )

    # stim → “input” neurons
    syn_input = Synapses(
        stim, neurons[0:n_input],
        'w : 1\nlayer : 1',
        on_pre='''
spikes_received += 1
sum += spike_timing(w, global_clock, layer, sum, spikes_received)

scheduled_time = (1/(1 + exp(-(sum/spikes_received))) + layer)*ms
'''
    )
    syn_input.connect(j='i')
    # syn_input.w = weights_1
    syn_input.layer = 0

    # “input” → hidden
    syn_hidden = Synapses(
        neurons[0:n_input], neurons[n_input:],
        'w : 1\nlayer : 1',
        on_pre='''
spikes_received += 1
sum += spike_timing(w, global_clock, layer, sum, spikes_received)
scheduled_time = (1/(1 + exp(-(sum/spikes_received))) + layer)*ms
'''
    )
    syn_hidden.connect()
    syn_hidden.w = weights_1
    syn_hidden.layer = 1

    # fire at scheduled_time
    neurons.run_regularly('''
v = int(abs(t - scheduled_time) < 0.0005*ms)*1.2
global_clock += 0.001
''', dt=0.001*ms)

    mon = SpikeMonitor(neurons)
    run(5*ms)

    # collect first spikes
    result = []
    for idx in range(n_total):
        ts = mon.spike_trains()[idx]
        result.append(round(float(ts[0]/ms),3) if len(ts) else None)
    return result

# Example call (2 inputs, 2→1 weights)
mini_urd_inputs    = np.array([0.4, 0.5]) # bias of .123
#mini_urd_weights_1 = np.array([1, 1])
mini_urd_weights_1 = np.array([.2, 1])
print(mini_urd(mini_urd_inputs, mini_urd_weights_1))

WARNING    Came across an abstract code block that may not be well-defined: the outcome may depend on the order of execution. You can ignore this warning if you are sure that the order of operations does not matter. 3 lines of abstract code, first line is: 'spikes_received += 1 (in-place)'
 [brian2.codegen.generators.base]
WARNING    Came across an abstract code block that may not be well-defined: the outcome may depend on the order of execution. You can ignore this warning if you are sure that the order of operations does not matter. 3 lines of abstract code, first line is: 'spikes_received += 1 (in-place)'
 [brian2.codegen.generators.base]


[0.599, 0.623, 1.697]


In [15]:
from brian2 import *
import numpy as np

start_scope()
defaultclock.dt = 0.0001*ms

# ---------------------------------------------------------------------
# 1) Your original spike_timing + its ∂/∂w
# ---------------------------------------------------------------------
@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def spike_timing(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    if w >= 0:
        return x**(1 - w)
    else:
        return 1 - (1 - x)**(1 + w)

@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def d_spike_timing_dw(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    # avoid log(0)
    eps = 1e-9
    if w >= 0:
        return - x**(1 - w) * np.log(x + eps)
    else:
        return - (1 - x)**(1 + w) * np.log(1 - x + eps)

# sigmoid derivative
def dsigmoid(z):
    s = 1.0/(1 + np.exp(-z))
    return s*(1 - s)

# ---------------------------------------------------------------------
# 2) Your mini_urd forward pass (unchanged)
# ---------------------------------------------------------------------
def mini_urd(inputs, weights_1):
    n_input  = 2
    n_hidden = 1
    n_total  = n_input + n_hidden

    neurons = NeuronGroup(
        n_total,
        '''
        v               : 1
        sum             : 1
        spikes_received : 1
        scheduled_time  : second
        global_clock    : 1
        ''',
        threshold='v > 1',
        reset='v = 0',
        method='exact'
    )
    neurons.v = 0
    neurons.sum = 0
    neurons.spikes_received = 0
    neurons.global_clock = 0
    neurons.scheduled_time = 1e9*second

    stim = SpikeGeneratorGroup(2,
        indices=[0,1],
        times=inputs*ms
    )

    # stim → input‐layer (no weights here)
    syn_input = Synapses(
        stim, neurons[0:n_input],
        'w : 1\nlayer : 1',
        on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, sum, spikes_received)
        scheduled_time = (1/(1 + exp(-(sum/spikes_received))) + layer)*ms
        '''
    )
    syn_input.connect(j='i')
    syn_input.layer = 0

    # input → hidden (this is the only trainable W)
    syn_hidden = Synapses(
        neurons[0:n_input], neurons[n_input:],
        'w : 1\nlayer : 1',
        on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, sum, spikes_received)
        scheduled_time = (1/(1 + exp(-(sum/spikes_received))) + layer)*ms
        '''
    )
    syn_hidden.connect()
    syn_hidden.w = weights_1
    syn_hidden.layer = 1

    # run neurons at their scheduled_time
    neurons.run_regularly('''
        v = int(abs(t - scheduled_time) < 0.0005*ms)*1.2
        global_clock += 0.001
    ''', dt=0.001*ms)

    mon = SpikeMonitor(neurons)
    run(5*ms)

    result = []
    for idx in range(n_total):
        ts = mon.spike_trains()[idx]
        if len(ts):
            result.append(float(ts[0]/ms))
        else:
            result.append(None)
    return result

# ---------------------------------------------------------------------
# 3) Training function for the 1‐layer mini model
# ---------------------------------------------------------------------
def train_mini_urd(training_data, target_times, w_init,
                   epochs=5, lr=0.1):
    """
    training_data: list of length-N samples, each is array([t0, t1])
    target_times:  list of length-N floats (desired hidden spike time in ms)
    w_init:        np.array of shape (2,)
    """
    w = w_init.copy()
    for ep in range(epochs):
        print(f"\n=== Epoch {ep+1}/{epochs} ===")
        for i, (inp, t_target) in enumerate(zip(training_data, target_times)):
            # ---- forward ----
            outs = mini_urd(inp, w)
            t_hidden = outs[-1]
            if t_hidden is None:
                # no spike → treat as very late
                t_hidden = 5.0

            # ---- loss & dL/dt ----
            # use ½*(t_h - t*)²
            loss = 0.5 * (t_hidden - t_target)**2
            dL_dt = (t_hidden - t_target)

            # ---- analytic backprop to w[0], w[1] ----
            # recompute sum and sr for hidden
            layer_h = 1
            # each input‐neuron spikes once, so sr = 2
            sr_h = 2.0
            # global_clock at arrival: use inp[i] ms for each
            s0 = spike_timing(w[0], inp[0], layer_h, 0, 1)
            s1 = spike_timing(w[1], inp[1], layer_h, 0, 1)
            sum_h = s0 + s1
            z = sum_h/sr_h

            # d t_sched / d sum_h
            # scheduled_time = (sigmoid(z) + layer_h) * ms
            # dt/dsum = ms * dsigmoid(z) * (1/sr_h)
            dt_dsum = dsigmoid(z) * (1/sr_h)

            # dsum/dw_i = d_spike_timing_dw(w_i, global_clock=inp[i], ...)
            dsum_dw = np.array([
                d_spike_timing_dw(w[0], inp[0], layer_h, 0, 1),
                d_spike_timing_dw(w[1], inp[1], layer_h, 0, 1),
            ])

            # full gradient: dL/dw_i = dL/dt * dt/dsum * dsum/dw_i
            grads = dL_dt * dt_dsum * dsum_dw

            # ---- print & update ----
            print(f"Sample {i}: inp={inp}, t_h={t_hidden:.3f}ms, target={t_target:.3f}ms, loss={loss:.4f}")
            print(" ∇w =", grads)
            w -= lr * grads

        print(" Updated w after epoch:", w)

    return w

# ---------------------------------------------------------------------
# 4) Quick demo
# ---------------------------------------------------------------------
if __name__ == "__main__":
    # four toy samples: two input‐spike times, and a target hidden time
    X = [np.array([0.2,0.5]),
         np.array([0.1,0.8]),
         np.array([0.7,0.3]),
         np.array([0.4,0.9])]
    Y = [1.0, 1.2, 1.8, 1.5]   # desired hidden spike times in ms

    w0 = np.array([1.0, -1.0])
    w_final = train_mini_urd(X, Y, w0, epochs=5, lr=0.2)

    print("\nFinal learned weights:", w_final)


=== Epoch 1/5 ===
Sample 0: inp=[0.2 0.5], t_h=1.622ms, target=1.000ms, loss=0.1934
 ∇w = [0.11762763 0.05065946]
Sample 1: inp=[0.1 0.8], t_h=1.728ms, target=1.200ms, loss=0.1394
 ∇w = [0.13643476 0.10232718]
Sample 2: inp=[0.7 0.3], t_h=1.493ms, target=1.800ms, loss=0.0471
 ∇w = [-0.0126795  -0.01305307]
Sample 3: inp=[0.4 0.9], t_h=1.726ms, target=1.500ms, loss=0.0255
 ∇w = [0.02357803 0.06605228]
 Updated w after epoch: [ 0.94700782 -1.04119717]

=== Epoch 2/5 ===
Sample 0: inp=[0.2 0.5], t_h=1.614ms, target=1.000ms, loss=0.1885
 ∇w = [0.10799947 0.0521212 ]
Sample 1: inp=[0.1 0.8], t_h=1.722ms, target=1.200ms, loss=0.1362
 ∇w = [0.12212377 0.11013685]
Sample 2: inp=[0.7 0.3], t_h=1.484ms, target=1.800ms, loss=0.0499
 ∇w = [-0.0128775  -0.01369549]
Sample 3: inp=[0.4 0.9], t_h=1.722ms, target=1.500ms, loss=0.0246
 ∇w = [0.02250171 0.0727253 ]
 Updated w after epoch: [ 0.89905832 -1.08545474]

=== Epoch 3/5 ===
Sample 0: inp=[0.2 0.5], t_h=1.605ms, target=1.000ms, loss=0.1830
 ∇w =

In [16]:
from brian2 import *
import numpy as np

start_scope()
defaultclock.dt = 0.0001*ms

# -----------------------------------------------------------------------------
# 1) spike_timing + its derivative
# -----------------------------------------------------------------------------
@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def spike_timing(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    if w >= 0:
        return x**(1 - w)
    else:
        return 1 - (1 - x)**(1 + w)

@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def d_spike_timing_dw(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    eps = 1e-9
    if w >= 0:
        return - x**(1 - w) * np.log(x + eps)
    else:
        return - (1 - x)**(1 + w) * np.log(1 - x + eps)

def dsigmoid(z):
    s = 1/(1 + np.exp(-z))
    return s*(1 - s)

# -----------------------------------------------------------------------------
# 2) mini_urd forward: returns hidden‐spike‐time only
# -----------------------------------------------------------------------------
def mini_urd(inputs, w):
    n_input  = 2
    n_hidden = 1
    n_total  = n_input + n_hidden

    G = NeuronGroup(
        n_total,
        '''
        v               : 1
        sum             : 1
        sr              : 1
        scheduled_time  : second
        global_clock    : 1
        ''',
        threshold='v>1', reset='v=0', method='exact'
    )
    G.v = 0; G.sum = 0; G.sr = 0
    G.global_clock = 0
    G.scheduled_time = 1e9*second

    stim = SpikeGeneratorGroup(2, indices=[0,1], times=inputs*ms)

    # first layer has fixed identity weights
    S1 = Synapses(stim, G[:2],
        'layer:1', on_pre='''
        sr += 1
        sum += spike_timing(1, global_clock, layer, sum, sr)
        scheduled_time = (1/(1+exp(-(sum/sr))) + layer)*ms
        '''
    )
    S1.connect(j='i')
    S1.layer = 0

    # trainable synapse 2→hidden
    S2 = Synapses(G[:2], G[2:],
        'w : 1\nlayer:1', on_pre='''
        sr += 1
        sum += spike_timing(w, global_clock, layer, sum, sr)
        scheduled_time = (1/(1+exp(-(sum/sr))) + layer)*ms
        '''
    )
    S2.connect()
    S2.w = w
    S2.layer = 1

    # drive v when scheduled_time hits
    G.run_regularly('''
        v = int(abs(t - scheduled_time)<0.0005*ms)*1.2
        global_clock += 0.001
    ''', dt=0.001*ms)

    mon = SpikeMonitor(G)
    run(5*ms)

    # return hidden spike time (or a large value if no spike)
    ts = mon.spike_trains()[2]
    return float(ts[0]/ms) if len(ts) else 5.0

# -----------------------------------------------------------------------------
# 3) Training with multi‐loss
# -----------------------------------------------------------------------------
def train_multi_loss(
    X,                    # list of np.array([t0,t1])
    t_hidden_targets,     # list of floats
    t0_targets,           # list of floats for s0
    t1_targets,           # list of floats for s1
    w_init,
    alpha=1.0, beta=1.0, gamma=1.0,
    epochs=5, lr=0.1
):
    """
    Multi-loss:
      L0 = ½ (s0 - t0)^2
      L1 = ½ (s1 - t1)^2
      Lf = ½ (t_h  - t_h*)^2
      L = α L0 + β L1 + γ Lf
    """
    w = w_init.copy()
    for ep in range(epochs):
        print(f"\n=== Epoch {ep+1}/{epochs} ===")
        for i, inp in enumerate(X):
            # forward pass
            t_h = mini_urd(inp, w)
            # recompute s0,s1 exactly the same way Brian did
            layer_h = 1
            # each input is first spike, so sr_i=1, sum_i=0 → use that
            s0 = spike_timing(w[0], inp[0], layer_h, 0, 1)
            s1 = spike_timing(w[1], inp[1], layer_h, 0, 1)

            # --- compute loss terms ---
            t0_tgt = t0_targets[i]
            t1_tgt = t1_targets[i]
            th_tgt = t_hidden_targets[i]

            L0 = 0.5*(s0 - t0_tgt)**2
            L1 = 0.5*(s1 - t1_tgt)**2
            Lf = 0.5*(t_h - th_tgt)**2
            L  = alpha*L0 + beta*L1 + gamma*Lf

            # --- gradients ---
            # ∂L0/∂w0 = (s0 - t0)*∂s0/∂w0
            dL0_dw0 = (s0 - t0_tgt) * d_spike_timing_dw(w[0], inp[0], layer_h, 0, 1)
            # ∂L1/∂w1
            dL1_dw1 = (s1 - t1_tgt) * d_spike_timing_dw(w[1], inp[1], layer_h, 0, 1)

            # ∂Lf/∂w0,w1 = ∂Lf/∂t_h × ∂t_h/∂sum × ∂sum/∂w_i
            dLf_dt  = (t_h - th_tgt)
            sum_tot = s0 + s1
            sr = 2.0
            z = sum_tot/sr
            dt_dsum = dsigmoid(z)*(1/sr)
            dsum_dw0 = d_spike_timing_dw(w[0], inp[0], layer_h, 0, 1)
            dsum_dw1 = d_spike_timing_dw(w[1], inp[1], layer_h, 0, 1)
            dLf_dw0 = dLf_dt * dt_dsum * dsum_dw0
            dLf_dw1 = dLf_dt * dt_dsum * dsum_dw1

            # combine
            grad0 = alpha*dL0_dw0 + gamma*dLf_dw0
            grad1 = beta *dL1_dw1 + gamma*dLf_dw1

            # print & update
            print(f"Sample {i}: inp={inp}, s0={s0:.3f}, s1={s1:.3f}, t_h={t_h:.3f}")
            print(f"  L0={L0:.4f}, L1={L1:.4f}, Lf={Lf:.4f}, L={L:.4f}")
            print(f"  ∇w = [{grad0:.5f}, {grad1:.5f}]")
            w[0] -= lr * grad0
            w[1] -= lr * grad1

        print(" Updated w:", w)

    return w

# -----------------------------------------------------------------------------
# 4) Demo
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    # toy data: 4 samples
    X = [np.array([0.2,0.5]),
         np.array([0.1,0.8]),
         np.array([0.7,0.3]),
         np.array([0.4,0.9])]

    # main target: hidden spike at these ms
    T_hidden = [1.0, 1.2, 1.8, 1.5]
    # aux targets for each synapse
    T0 = [0.3, 0.2, 0.6, 0.5]
    T1 = [0.7, 0.8, 0.4, 0.6]

    w0 = np.array([0.2, 1.0])
    w_final = train_multi_loss(X, T_hidden, T0, T1, w0,
                               alpha=1.0, beta=1.0, gamma=0.5,
                               epochs=5, lr=0.1)

    print("\nFinal weights:", w_final)


=== Epoch 1/5 ===
Sample 0: inp=[0.2 0.5], s0=0.276, s1=1.000, t_h=1.709
  L0=0.0003, L1=0.0450, Lf=0.2513, L=0.1710
  ∇w = [0.00712, 0.23573]
Sample 1: inp=[0.1 0.8], s0=0.158, s1=0.995, t_h=1.685
  L0=0.0009, L1=0.0190, Lf=0.1176, L=0.0786
  ∇w = [-0.00504, 0.04943]
Sample 2: inp=[0.7 0.3], s0=0.752, s1=0.966, t_h=1.708
  L0=0.0115, L1=0.1603, Lf=0.0042, L=0.1739
  ∇w = [0.03938, 0.65315]
Sample 3: inp=[0.4 0.9], s0=0.479, s1=0.990, t_h=1.685
  L0=0.0002, L1=0.0761, Lf=0.0171, L=0.0849
  ∇w = [-0.00493, 0.04176]
 Updated w: [0.19634647 0.90199285]

=== Epoch 2/5 ===
Sample 0: inp=[0.2 0.5], s0=0.274, s1=0.934, t_h=1.706
  L0=0.0003, L1=0.0275, Lf=0.2492, L=0.1524
  ∇w = [0.00647, 0.17787]
Sample 1: inp=[0.1 0.8], s0=0.157, s1=0.974, t_h=1.685
  L0=0.0009, L1=0.0152, Lf=0.1176, L=0.0750
  ∇w = [-0.00544, 0.04403]
Sample 2: inp=[0.7 0.3], s0=0.751, s1=0.865, t_h=1.705
  L0=0.0114, L1=0.1082, Lf=0.0045, L=0.1219
  ∇w = [0.03901, 0.47942]
Sample 3: inp=[0.4 0.9], s0=0.477, s1=0.982, t_h

In [4]:
from brian2 import *
import numpy as np
import warnings

#warnings.filterwarnings("ignore", category=UserWarning, module='brian2.codegen.generators.base')

start_scope()

defaultclock.dt = 0.0001*ms  

# Custom timing function
@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, result=1, sum=1, spikes_received=1)
def spike_timing(w, global_clock, layer, sum, spikes_received): 
    x = global_clock % 1
    if w >= 0:
        return (x ** (1 - w)) 
    else:
        return (1 - (1 - x) ** (1 + w)) 
    
@implementation('numpy', discard_units=True)
@check_units(layer=1, result=1, sum=1, spikes_received=1)
def math1(layer, sum, spikes_received): 
    return (sum/spikes_received )+ layer

def run_Urd(inputs, weights_1, weights_2, weights_3):
    '''4-10-3 SNN'''
    # will add check of weights # so it all works
    n_input = 4 
    n_hidden = 10
    n_output = 3
    n_total = n_input + n_hidden + n_output

    neurons = NeuronGroup(n_total, '''
        v : 1
        sum : 1
        spikes_received : 1
        scheduled_time : second
        global_clock : 1
    ''', threshold='v > 1', reset='v = 0', method='exact')
    neurons.v = 0
    neurons.scheduled_time = 1e9 * second
    neurons.global_clock = 0.0
    neurons.sum = 0.0
    neurons.spikes_received = 0.0


    indicess = [i for i in range(n_input)]
    stim = SpikeGeneratorGroup(n_input, indices=indicess, times=(inputs*ms))

    syn_input = Synapses(stim, neurons[0:n_input], '''
        w : 1
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
    ''')
    syn_input.connect(j='i')
    syn_input.w = weights_1
    syn_input.layer = 0

    syn_hidden = Synapses(neurons[0:n_input], neurons[n_input:n_input+n_hidden], '''
        w : 1
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
    ''')
    for inp in range(n_input):
        for hid in range(n_hidden):
            syn_hidden.connect(i=inp, j=hid)

    syn_hidden.w = weights_2
    syn_hidden.layer = 1


    syn_output = Synapses(
        neurons[n_input:n_input+n_hidden], 
        neurons[n_input+n_hidden:n_total], 
        '''
        w : 1
        layer : 1
        ''',
        on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
        '''
    )

    for hid in range(n_hidden):
        for out in range(n_output):
            syn_output.connect(i=hid, j=out)

    # Set weights in correct order
    syn_output.w[:] = weights_3
    syn_output.layer = 2

    #print(syn_output.i[:], syn_output.j[:])
    #weights_into_output_1 = weights_3[1::3]



    neurons.run_regularly('''
        v = int(abs(t - scheduled_time) < 0.0005*ms) * 1.2
        global_clock += 0.001
    ''', dt=0.001*ms)


    spikemon = SpikeMonitor(neurons)
    
    # neurons.v = 0
    # neurons.scheduled_time = 1e9 * second
    # neurons.global_clock = 0.0
    # neurons.sum = 0.0
    # neurons.spikes_received = 0.0

    run(5*ms)

    result = []

    for i in range(n_total):
        times = spikemon.spike_trains()[i]
        if len(times) > 0:
            result.append(round(times[0]/ms, 3))
        else:
            result.append(None)  # or some other placeholder like float('nan')
            
    return result




In [12]:
import numpy as np
import logging

logging.getLogger('brian2').setLevel(logging.ERROR)


import numpy as np
from copy import deepcopy

def compute_loss(predicted, desired):
    """
    MSE loss on the 3 output spike-times.
    predicted: full list of 17 times (None → we treat as large penalty)
    desired: array-like of length 3
    """
    # pull out last 3 spikes
    out_t = np.array([t if t is not None else 10.0 for t in predicted[-3:]])
    d = np.array(desired)
    return np.mean((out_t - d)**2)

def finite_difference_grads(inputs, w1, w2, w3, desired, eps=3):
    """
    Returns numerical gradients of loss wrt each weight array.
    """
    # baseline loss
    base_out = run_Urd(inputs, w1, w2, w3)
    L0 = compute_loss(base_out, desired)

    # grad for layer1
    gw1 = np.zeros_like(w1)
    for idx in np.ndindex(w1.shape):
        w1p = w1.copy()
        w1p[idx] += eps
        Lp = compute_loss(run_Urd(inputs, w1p, w2, w3), desired)
        gw1[idx] = (Lp - L0)/eps

    # grad for layer2
    gw2 = np.zeros_like(w2)
    for idx in np.ndindex(w2.shape):
        w2p = w2.copy()
        w2p[idx] += eps
        Lp = compute_loss(run_Urd(inputs, w1, w2p, w3), desired)
        gw2[idx] = (Lp - L0)/eps

    # grad for layer3
    gw3 = np.zeros_like(w3)
    for idx in np.ndindex(w3.shape):
        w3p = w3.copy()
        w3p[idx] += eps
        Lp = compute_loss(run_Urd(inputs, w1, w2, w3p), desired)
        gw3[idx] = (Lp - L0)/eps

    return gw1, gw2, gw3, L0

def backprop_snn_fd(inputs, w1, w2, w3,
                    desired=[2.1, 2.6, 2.9],
                    lr=0.1, eps=3):
    """
    One gradient‐step on the SNN via finite‐difference.
    Returns updated (w1, w2, w3) and the loss before update.
    """
    gw1, gw2, gw3, loss = finite_difference_grads(inputs, w1, w2, w3, desired, eps)
    # gradient descent
    w1 -= lr * gw1
    w2 -= lr * gw2
    w3 -= lr * gw3
    return w1, w2, w3, loss

# — example usage —
# random init
inputs    = np.random.uniform(0, 1, 4)
weights_1 = np.random.uniform(0, 1, 4)
weights_2 = np.random.uniform(0, 1, 40)
weights_3 = np.random.uniform(0, 1, 30)

print(weights_1, weights_2, weights_3)

desired = [2.1, 2.6, 2.9]
for epoch in range(1):
    w1, w2, w3, loss = backprop_snn_fd(
        inputs, weights_1, weights_2, weights_3,
        desired=desired, lr=0.5, eps=3
    )
    weights_1, weights_2, weights_3 = w1, w2, w3
    out = run_Urd(inputs, w1, w2, w3)[-3:]
    print(f"Epoch {epoch:2d}  Loss={loss:.4f}  Outputs={out}")

print(weights_1, weights_2, weights_3)

# After a few epochs you should see the 3 output times marching
# closer to [2.1, 2.6, 2.9].

[0.14345227 0.86066371 0.88725555 0.85814195] [0.44777422 0.76353581 0.48259652 0.3623195  0.99852955 0.99073865
 0.90408036 0.39032022 0.29293302 0.22087558 0.15663    0.32492888
 0.56082809 0.25731051 0.04950147 0.28284291 0.23587225 0.44035401
 0.39615996 0.35721367 0.89914801 0.44307031 0.63992119 0.52551554
 0.98827489 0.05136685 0.22214933 0.74473358 0.12725664 0.83123581
 0.98797027 0.54554983 0.49111424 0.37928825 0.2765512  0.63761703
 0.15451435 0.66322082 0.94605562 0.18985174] [0.22203751 0.43385482 0.91329917 0.06779644 0.40236334 0.81143013
 0.4022748  0.1185883  0.60056945 0.44718627 0.4049495  0.8794631
 0.85964283 0.19174956 0.60335014 0.80864427 0.46067967 0.14843805
 0.04586898 0.5629214  0.93889325 0.55204879 0.64219171 0.40427682
 0.87504725 0.92320919 0.09731941 0.36464712 0.61217678 0.59806614]
Epoch  0  Loss=0.2828  Outputs=[2.948, 2.955, 2.962]
[0.13953711 0.86182532 0.88831144 0.85949145] [0.44743883 0.76336731 0.48216986 0.36189828 0.99909788 0.9911931
 0.903

Epoch 1/20 – Avg spikes: 0.00
Epoch 2/20 – Avg spikes: 0.00
Epoch 3/20 – Avg spikes: 0.00
Epoch 4/20 – Avg spikes: 0.00
Epoch 5/20 – Avg spikes: 0.00
Epoch 6/20 – Avg spikes: 0.00
Epoch 7/20 – Avg spikes: 0.00
Epoch 8/20 – Avg spikes: 0.00
Epoch 9/20 – Avg spikes: 0.00
Epoch 10/20 – Avg spikes: 0.00
Epoch 11/20 – Avg spikes: 0.00
Epoch 12/20 – Avg spikes: 0.00
Epoch 13/20 – Avg spikes: 0.00
Epoch 14/20 – Avg spikes: 0.00
Epoch 15/20 – Avg spikes: 0.00
Epoch 16/20 – Avg spikes: 0.00
Epoch 17/20 – Avg spikes: 0.00
Epoch 18/20 – Avg spikes: 0.00
Epoch 19/20 – Avg spikes: 0.00
Epoch 20/20 – Avg spikes: 0.00
Final first-spike times (ms): [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
